# Day-2 GPU entropy sieve (Colab)

This notebook runs the repository's `sieve.py`, auto-selecting GPU via CuPy when available and falling back to NumPy on CPU.

**Outputs**:
- CSV summary (per trace)
- JSON meta summary (np-wall hit %, thresholds, run params)


In [ ]:
# Clone repo into /content if needed (Colab) and cd into it
import os, sys, subprocess, pathlib
COLAB = bool(os.environ.get('COLAB_GPU') or 'google.colab' in sys.modules)
repo_url = 'https://github.com/derekwins88/Brain.git'
work = pathlib.Path('/content/Brain' if COLAB else pathlib.Path.cwd())
if COLAB:
    if not work.exists():
        subprocess.run(['git', 'clone', repo_url, str(work)], check=True)
    os.chdir(work)
print('cwd =', os.getcwd())
!git rev-parse --short HEAD || true

In [ ]:
# Install Python deps (quiet). Try CuPy first (GPU), fall back to NumPy.
import sys, subprocess
def pip(*args):
    return subprocess.run([sys.executable, '-m', 'pip', 'install', *args], check=False)
print('Installing NumPy …')
pip('-q', 'numpy>=1.26')
print('Trying CuPy (CUDA 12.x) …')
pip('-q', 'cupy-cuda12x>=13.0')
print('Deps done.')

In [ ]:
# Configure run parameters via environment variables (edit as needed)
import os
os.environ.setdefault('N_TRACES', '2000')       # quick sample for Colab demo
os.environ.setdefault('TRACE_LEN', '48')
os.environ.setdefault('CSV_PATH', 'data/entropy_sieve_ci.csv')
os.environ.setdefault('META_PATH', 'data/entropy_sieve_ci.json')
os.environ.setdefault('NP_THRESHOLD', '0.09')
os.environ.setdefault('IRREV_EPS', '0.045')
print({k: os.environ[k] for k in ['N_TRACES','TRACE_LEN','CSV_PATH','META_PATH','NP_THRESHOLD','IRREV_EPS']})

In [ ]:
# Run the sieve
import sys, subprocess
ret = subprocess.run([sys.executable, 'sieve.py'])
print('sieve.py exit code:', ret.returncode)

In [ ]:
# Preview outputs
import os, json, pandas as pd
csv_path = os.environ['CSV_PATH']
meta_path = os.environ['META_PATH']
print('\nCSV head:')
try:
    import pandas as pd
    display(pd.read_csv(csv_path).head())
except Exception as e:
    print('CSV preview skipped:', e)
print('\nMeta JSON:')
try:
    print(json.dumps(json.load(open(meta_path)), indent=2))
except Exception as e:
    print('Meta preview skipped:', e)

### Save artifacts
- Files are written into the repo's `data/` folder.
- In Colab, use *File → Download* to pull them locally, or push them back to GitHub from a local machine.